In [1]:
import pandas as pd
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelBinarizer
import numpy as np

Dataset modificado
Obs: lembrar de por o original, e de incluir o codigo de mudar os dados

In [2]:
data = pd.read_csv("train.csv", sep=",")

In [9]:
data.shape

(878049, 8)

In [8]:
data.head()

,Year,Month,Day,Hour,Category,DayOfWeek,X,Y
0,2015,5,13,23,WARRANTS,Wednesday,-122.425892,37.774599
1,2015,5,13,23,OTHER OFFENSES,Wednesday,-122.425892,37.774599
2,2015,5,13,23,OTHER OFFENSES,Wednesday,-122.424363,37.800414
3,2015,5,13,23,LARCENY/THEFT,Wednesday,-122.426995,37.800873
4,2015,5,13,23,LARCENY/THEFT,Wednesday,-122.438738,37.771541


In [10]:
np.unique(data["Category"])

array(['ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY',
       'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE',
       'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION',
       'FAMILY OFFENSES', 'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING',
       'KIDNAPPING', 'LARCENY/THEFT', 'LIQUOR LAWS', 'LOITERING',
       'MISSING PERSON', 'NON-CRIMINAL', 'OTHER OFFENSES',
       'PORNOGRAPHY/OBSCENE MAT', 'PROSTITUTION', 'RECOVERED VEHICLE',
       'ROBBERY', 'RUNAWAY', 'SECONDARY CODES', 'SEX OFFENSES FORCIBLE',
       'SEX OFFENSES NON FORCIBLE', 'STOLEN PROPERTY', 'SUICIDE',
       'SUSPICIOUS OCC', 'TREA', 'TRESPASS', 'VANDALISM', 'VEHICLE THEFT',
       'WARRANTS', 'WEAPON LAWS'], dtype=object)

In [11]:
#Number of instances per class
data.groupby('Category')['Category'].count().sort_values(ascending=True)

Category
TREA                                6
PORNOGRAPHY/OBSCENE MAT            22
GAMBLING                          146
SEX OFFENSES NON FORCIBLE         148
EXTORTION                         256
BRIBERY                           289
BAD CHECKS                        406
FAMILY OFFENSES                   491
SUICIDE                           508
EMBEZZLEMENT                     1166
LOITERING                        1225
ARSON                            1513
LIQUOR LAWS                      1903
RUNAWAY                          1946
DRIVING UNDER THE INFLUENCE      2268
KIDNAPPING                       2341
RECOVERED VEHICLE                3138
DRUNKENNESS                      4280
DISORDERLY CONDUCT               4320
SEX OFFENSES FORCIBLE            4388
STOLEN PROPERTY                  4540
TRESPASS                         7326
PROSTITUTION                     7484
WEAPON LAWS                      8555
SECONDARY CODES                  9985
FORGERY/COUNTERFEITING          10609
FRA

In [3]:
df = data[['Category', 'X', 'Y']]
df.head()


,Category,X,Y
0,WARRANTS,-122.425892,37.774599
1,OTHER OFFENSES,-122.425892,37.774599
2,OTHER OFFENSES,-122.424363,37.800414
3,LARCENY/THEFT,-122.426995,37.800873
4,LARCENY/THEFT,-122.438738,37.771541


Treino de random forest pelo esquema 80 treino 20 teste

In [31]:
X_train, X_test, y_train, y_test = train_test_split(df[["X","Y"]], df["Category"], test_size=0.2)
scaler = MinMaxScaler()

In [32]:
clf = make_pipeline(MinMaxScaler(), RandomForestClassifier(n_estimators=150,  max_depth=20, n_jobs=4))

In [33]:
clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split...n_jobs=4,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [34]:
pred = clf.predict_proba(X_test)

In [17]:
pred.shape

(10, 9)

In [ ]:
log_loss(y_test, pred)

Ajustando o modelo de random forest no esquema 3-fold cross validation utilizando busca em grid.
Aplica scaling de 0 a 1 nos dados previamente

In [4]:
from model_tuning import ClassifierPipeline

Atentar que o output da funcao eh o log_loss negado pq a busca em grid sempre maximiza o valor do score por default, portanto valores que devem ser minimizados possuem seus valores negados.

In [6]:
rf_parameters = {
        "n_estimators":[150, 200, 250],
        "max_depth": [15, 20, 30],
        "max_features": ["sqrt", None]
}
pipeline = ClassifierPipeline(RandomForestClassifier(), rf_parameters, n_jobs=2)
pipeline.fit(df[["X","Y"]], df["Category"])


Best score: -2.479
Best parameters set:
	clf__max_depth: 15
	clf__max_features: 'sqrt'
	clf__n_estimators: 250


In [6]:
pipeline.get_cv_results()

/home/andrea/Workspace/kaggle/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/andrea/Workspace/kaggle/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/andrea/Workspace/kaggle/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/andrea

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__n_estimators,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,mean_train_score,std_train_score
0,0.028312,0.007119,0.003015,0.000455,20,{'clf__n_estimators': 20},-8.621817,-2.917869,-5.836168,2.851203,1,-0.326394,-0.27258,-0.299487,0.026907
